# HW 6.1

In this part, we only need to scrap the video's url, video ID, author, and total views from the trending website. To avoiding the banning of the Youtube, more data will extract by Youtube API after saving all trending page information for 7 days. So, we will run this simple code every day for a week, and this code is an example. We are also working on the API. 

In [1]:

import warnings
import requests
import json
import time
import codecs
import re
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def webcapture(url):
    # Scrap the page in url, return Soup object
    headers = {
   'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
   }
    
    data = requests.get(url, headers=headers) 
    soup = BeautifulSoup(data.text, 'html.parser') 
    return soup 
trending_url="https://www.youtube.com/feed/trending"
soup=webcapture(trending_url)

In [4]:
#create files
with open('trending.hml', "w",encoding='utf-8') as file:
    file.write(str(soup))
    file.close()

with open('trending.hml',encoding="utf-8") as page:
  #parse as beautifulsoup object
  soup=BeautifulSoup(page, 'html.parser')

In [5]:
# Extract data
soup=str(soup)
url_list=re.findall('"url":"/watch\\?v\\=.{12}',soup) 
views = re.findall('"shortViewCountText":{"simpleText":"(.+?) views"},',soup)
author = re.findall('"ownerText":{"runs":.{"text":"(.+?)",',soup)
video_id = re.findall('"watchEndpoint":{"videoId":"(.+?)"}}',soup)

In [6]:
# Edit url into links
for i in range(len(url_list)):
    url_list[i]=url_list[i].replace('"url":"',"http://www.youtube.com").replace('"','')

In [7]:
# Create a zipped list of tuples from above lists
zippedList =  list(zip(video_id,author,views,url_list))
# Create a dataframe from zipped list
trending_df = pd.DataFrame(zippedList, columns = ['video_id' , 'author', 'views','url_list'])

In [10]:
trending_df.head(10)

,video_id,author,views,url_list
0,FZV6Hb7-d4E,How Ridiculous,1.1M,http://www.youtube.com/watch?v=FZV6Hb7-d4E
1,tMKXbLBgkEc,Mark Rober,3.3M,http://www.youtube.com/watch?v=tMKXbLBgkEc
2,0jz0GAFNNIo,Drake,8.4M,http://www.youtube.com/watch?v=0jz0GAFNNIo
3,zFjdfWfg--M,Bad Bunny,229K,http://www.youtube.com/watch?v=zFjdfWfg--M
4,MtK36uCmDtQ,LIL UZI VERT,23K,http://www.youtube.com/watch?v=MtK36uCmDtQ
5,L9RYLPgNzys,ESPN,10K,http://www.youtube.com/watch?v=L9RYLPgNzys
6,UyFfFXrp9sc,The Royalty Family,4.6K,http://www.youtube.com/watch?v=UyFfFXrp9sc
7,JSAvt7xsizU,beIN SPORTS USA,3.9K,http://www.youtube.com/watch?v=JSAvt7xsizU
8,kKdtU4dnnQY,Breakfast Club Power 105.1 FM,5.1K,http://www.youtube.com/watch?v=kKdtU4dnnQY
9,CPK_IdHe1Yg,Good Morning America,5.1M,http://www.youtube.com/watch?v=CPK_IdHe1Yg


In [9]:
# save to csv
trending_df.to_csv('trending_mar_2.csv',index=False)

## HW 6.2  
This assignment builds upon assignment 5.2.  In the last assignment, you have developed code that reads and uploads part of “reviews_electronics.16.json” to a MongoDB database called “amazon”.

a) Augment last week’s code to create a collection called “electronics” in “amazon” that contains each line of “reviews_electronics.16.json” as an individual document.
Write code that reads all documents in “electronics”, and prints the first 25 documents to screen.

- Please ignore the field “reviewText” for all subsequent questions as it would only eat up space.  Again, “reviewText” does not need to be part of your consideration. 

In [0]:
import pymongo
import json
import pprint

***Step 1: Connect to MongoDB, create the DB & collection***

In [0]:
# connect to mongoDB
client = pymongo.MongoClient()
# create the database
db_name = "amazon"
if db_name not in client.list_database_names():
    db = client[db_name]
    print("database created")
else:
    print("database exist")
    drop = input("Drop the database? Y/N") 
    if drop == "Y":
        client.drop_database(db_name) 

database created


In [0]:
# create the collection
collection = db["electronics"]

***Step 2: Import Json file to the collection***

In [0]:
# read Json file by line into a list
file_data = []
with open('reviews_electronics.16.json', 'r', encoding="UTF-8") as f:
    for line in f:
        file_data.append(json.loads(line))
file_data[:10]

[{'reviewerID': 'AKM1MP6P0OYPR',
  'asin': '0132793040',
  'reviewerName': 'Vicki Gibson "momo4"',
  'helpful': [1, 1],
  'reviewText': 'Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.',
  'overall': 5.0,
  'summary': 'Very thorough',
  'unixReviewTime': 1365811200,
  'reviewTime': '04 13, 2013'},
 {'reviewerID': 'A2X8VX4DPMQFQQ',
  'asin': 'B00E4KP4W6',
  'reviewerName': 'lily68',
  'helpful': [1, 1],
  'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier to apply than the films too!",
  'overall': 5.0,
  'summary': 'LOVE this screen protector!!',

In [0]:
# insert documents into Mongo DB
collection.insert_many(file_data)
print("There are {} number of reviews ".format(collection.count_documents({})))

There are 324546 number of reviews 


***Step 3: Reads all documents in “electronics” & prints the first 25 documents***

In [0]:
# read all docs
all_documents = db.electronics.find({})
# print the first 25 docs
for document in all_documents.limit(25): 
    pprint.pprint(document)

{'_id': ObjectId('5e5de768e9e2b9b2ef053ca9'),
 'asin': '0132793040',
 'helpful': [1, 1],
 'overall': 5.0,
 'reviewText': 'Corey Barker does a great job of explaining Blend Modes in '
               'this DVD. All of the Kelby training videos are great but '
               'pricey to buy individually. If you really want bang for your '
               'buck just subscribe to Kelby Training online.',
 'reviewTime': '04 13, 2013',
 'reviewerID': 'AKM1MP6P0OYPR',
 'reviewerName': 'Vicki Gibson "momo4"',
 'summary': 'Very thorough',
 'unixReviewTime': 1365811200}
{'_id': ObjectId('5e5de768e9e2b9b2ef053caa'),
 'asin': 'B00E4KP4W6',
 'helpful': [1, 1],
 'overall': 5.0,
 'reviewText': "I can't believe I waited to long to switch to a glass screen "
               'protector.  I love this.  It feels and looks like there is no '
               'protector on.  It does show fingerprints, which I think is '
               'inevitable unless you use a matte finish screen protector, but '
             

b) Using Python or Java: Create a SQL database named “amazon_SQL”, In this database, create a SQL table named “electronics_SQL” that can hold each document from “electronics”.  Please chose your data types wisely.

Develop code (in Python or Java) that (1) reads all documents from “electronics” in MongoDB, and (2) writes that content to “electronics_SQL” in SQL. Print the first 25 documents to the screen. 

In [0]:
import mysql.connector
import re
import prettytable

In [0]:
DB_CONN = mysql.connector.connect(host='localhost',
                                  user='root',
                                  password='123456qwerty',
                                  port=3306,
                                  auth_plugin='mysql_native_password')

DB_NAME = 'amazon_SQL'
TABLE_NAME = 'electronics_SQL'

TABLE_SCHEMA = "(" \
                "_id VARCHAR(500) NOT NULL," \
                "asin VARCHAR(500)," + \
                "helpful_start SMALLINT(100)," + \
                "helpful_end SMALLINT(100)," + \
                "overall SMALLINT(100)," + \
                "reviewTime DATE," + \
                "reviewerID VARCHAR(500)," + \
                "reviewerName VARCHAR(500)," + \
                "summary VARCHAR(500)," + \
                "unixReviewTime VARCHAR(500)," + \
                "PRIMARY KEY (_id)" \
                ")"

***Step 1: Create mysql database and table***

In [0]:
def create_db_and_table_if_not_exists(DB_NAME, TABLE_NAME, TABLE_SCHEMA):
    try:
        
        cursor = DB_CONN.cursor()
        
        # Create the DB if not exists
        create_db_operation = 'CREATE DATABASE IF NOT EXISTS ' + DB_NAME
        cursor.execute(create_db_operation)
        
        # Select the database just created
        DB_CONN.database = DB_NAME

        # Create the table if not exists
        create_table_operation = "CREATE TABLE IF NOT EXISTS " + DB_NAME + "." + TABLE_NAME + " " + TABLE_SCHEMA + ";"
        cursor.execute(create_table_operation)
        
        # Close cursor
        cursor.close()
        return
    
    except IOError as e:
        print("Failed to create database or table.", e)

In [0]:
create_db_and_table_if_not_exists(DB_NAME, TABLE_NAME, TABLE_SCHEMA)

***Step2: Clean data***

In [0]:
# reads all documents from “electronics” in MongoDB into a nested list
all_documents = db.electronics.find({})

electronics_nested_list = []

for document in all_documents:
    
    _id = str(document.get("_id"))
    asin = document.get("asin")
    helpful_start = document.get("helpful")[0]
    helpful_end = document.get("helpful")[1]
    overall = document.get("overall")
    
    # convert reviewTime to DATE format
    reviewTime = document.get("reviewTime")
    reviewTime_date = reviewTime[-4:] + "-" + reviewTime[0:2] + "-" + re.findall(r' ([0-9]+)\,', reviewTime)[0]

    reviewerID = document.get("reviewerID")
    reviewerName = document.get("reviewerName")
    summary = document.get("summary")
    unixReviewTime = document.get("unixReviewTime")
    
    doc_list = [_id, asin, helpful_start, helpful_end, overall, reviewTime_date, reviewerID, reviewerName, summary, unixReviewTime]
    electronics_nested_list.append(doc_list)


***Step 3: Insert data into MySQL database***

In [0]:
insert_query = "INSERT INTO " + TABLE_NAME + \
                   "(_id, asin, helpful_start, helpful_end, overall, reviewTime, reviewerID, reviewerName, summary, unixReviewTime)" + \
                   " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"

In [0]:
def insert_nested_list_into_MYSQL_table(DB_NAME, INSERT_QUERY, NESTED_LIST):
    
    try:
        cursor = DB_CONN.cursor()
        DB_CONN.database = DB_NAME
        
        cursor.executemany(INSERT_QUERY, NESTED_LIST)
        DB_CONN.commit()
        cursor.close()

    except IOError as e:
        print(e)

In [0]:
insert_nested_list_into_MYSQL_table(DB_NAME, insert_query, electronics_nested_list)

***Step 4: Query the first 25 records***

In [0]:
cursor = DB_CONN.cursor()
DB_CONN.database = DB_NAME

query = 'SELECT * FROM electronics_SQL LIMIT 25;'
cursor.execute(query) 

# Use prettytable to pretty print the sql result
titles = [title[0] for title in cursor.description]
table = prettytable.PrettyTable(titles)
for row in cursor.fetchall():
    table.add_row(row)
print(table)

+--------------------------+------------+---------------+-------------+---------+------------+----------------+-----------------------------+---------------------------------------------------------------------------------------------------------+----------------+
|           _id            |    asin    | helpful_start | helpful_end | overall | reviewTime |   reviewerID   |         reviewerName        |                                                 summary                                                 | unixReviewTime |
+--------------------------+------------+---------------+-------------+---------+------------+----------------+-----------------------------+---------------------------------------------------------------------------------------------------------+----------------+
| 5e5de768e9e2b9b2ef053ca9 | 0132793040 |       1       |      1      |    5    | 2013-04-13 | AKM1MP6P0OYPR  |     Vicki Gibson "momo4"    |                                              Very thorough     

In [0]:
DB_CONN.close()